In [35]:
# !pip install -r ../exp_requirements.txt

In [36]:
# !pip uninstall virny -y

In [37]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@development

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@development

In [14]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

# Folktables GA Dataset With Random Nulls

## Import dependencies

In [16]:
import os
import copy

from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from virny.utils.custom_initializers import create_config_obj, create_models_metrics_dct_from_database_df
from virny.datasets.data_loaders import ACSEmploymentDataset
from source.user_interfaces.experiment_interface import run_exp_iter_with_models_stress_testing
from source.error_injectors.random_nulls_injector_v2 import RandomNullsInjectorV2
from source.utils.custom_initializers import create_experiment_data_loader
from source.utils.db_functions import read_model_metric_dfs_from_db
from source.preprocessing.basic_preprocessing import get_null_imputer_preprocessor

from configs.constants import NUM_METRICS_COMPUTATION_RUNS, EXPERIMENT_SEEDS, TEST_SET_FRACTION

## Initialize input variables for the experiment

In [17]:
ROOT_DIR = os.path.join(os.getcwd(), "..", "..")
EXPERIMENT_NAME = 'stress_testing_nulls'
DB_COLLECTION_NAME = f'exp_{EXPERIMENT_NAME}'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)
COLUMNS_TO_TRANSFORM = ['MAR', 'MIL', 'ESP', 'MIG', 'DREM', 'NATIVITY', 'DIS', 'DEAR', 'DEYE', 'RELP', 'CIT', 'ANC','SCHL']
MAX_NUM_COLUMNS_TO_EFFECT = 4
INJECTOR_CONFIG_LST = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
# INJECTOR_CONFIG_LST = [0.1, 0.2, 0.4, 0.6, 0.8, 1.0]

In [18]:
data_loader = ACSEmploymentDataset(state=['GA'], year=2018, with_nulls=False, subsample_size=50_000, subsample_seed=42)
# data_loader = ACSEmploymentDataset(state=['GA'], year=2018, with_nulls=False)
data_loader.X_data.head()

,MAR,MIL,ESP,MIG,DREM,NATIVITY,DIS,DEAR,DEYE,SEX,RAC1P,RELP,CIT,ANC,SCHL,AGEP
0,1,4,0,1,2,1,2,2,2,2,1,1,1,1,21,31
1,5,4,0,3,2,2,2,2,2,2,1,12,5,2,21,22
2,2,4,0,1,2,1,1,2,1,2,1,16,1,1,19,88
3,1,4,0,1,2,1,2,2,2,2,2,0,1,1,16,71
4,1,2,0,1,1,1,1,1,2,1,1,0,1,1,21,44


In [19]:
data_loader.full_df.shape

(50000, 16)

In [123]:
data_loader.full_df.isna().sum()

MAR         0
MIL         0
ESP         0
MIG         0
DREM        0
NATIVITY    0
DIS         0
DEAR        0
DEYE        0
SEX         0
RAC1P       0
RELP        0
CIT         0
ANC         0
SCHL        0
AGEP        0
dtype: int64

### Define a db writer and custom fields to insert into your database

In [20]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [21]:
custom_table_fields_dct = {
    'error_type': 'Random Nulls',
    'error_percentages': '0%,5%,10%,20%,30%,40%,50%',
    # 'error_percentages': '0%,10%,20%,40%,60%,80%,100%',
    # 'preprocessing_technique': 'cat: mode_trimmed_0.3, num: median',
    'preprocessing_technique': 'cat: mode, num: median',
}

In [23]:
import uuid

# custom_table_fields_dct['session_uuid'] = 'c53d250b-5ba9-4d91-a444-ed7eb7919de5'
# custom_table_fields_dct['session_uuid'] = 'e38b6e20-e4e7-4791-89d3-5e371513d2ea'
# custom_table_fields_dct['session_uuid'] = '2f6541f0-2a77-446a-a5be-92d7b4187052'
# custom_table_fields_dct['session_uuid'] = 'a4febf95-81bf-4aee-9b51-a690d596d5d2'

custom_table_fields_dct['session_uuid'] = '0fe1a14e-49c1-4764-962a-b50a2fbed1b7'
# custom_table_fields_dct['session_uuid'] = str(uuid.uuid4())
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  0fe1a14e-49c1-4764-962a-b50a2fbed1b7


### Create a metrics computation config object

In [24]:
config_yaml_path = 'experiment_config.yaml'
age_range = [i for i in range(30, 41)]
config_yaml_content = \
f"""
dataset_name: Folktables_Employment_GA_2018
bootstrap_fraction: 0.8
# n_estimators: 50
n_estimators: 10
# num_runs: {NUM_METRICS_COMPUTATION_RUNS}
num_runs: 2
sensitive_attributes_dct: {{'SEX': '1', 'RAC1P': '1', 'AGEP': {age_range}, 'SEX & RAC1P & AGEP': None}}
"""

with open(config_yaml_path, 'w', encoding='utf-8') as f:
    f.write(config_yaml_content)

In [25]:
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

### Define tuning parameter for models

In [26]:
def get_models_params_for_tuning(models_tuning_seed):
    return {
        'DecisionTreeClassifier': {
            'model': DecisionTreeClassifier(random_state=models_tuning_seed),
            'params': {
                "max_depth": [20, 30],
                "min_samples_split" : [0.1],
                "max_features": ['sqrt'],
                "criterion": ["gini", "entropy"]
            }
        },
        'LogisticRegression': {
            'model': LogisticRegression(random_state=models_tuning_seed),
            'params': {
                'penalty': ['l2'],
                'C' : [0.0001, 0.1, 1, 100],
                'solver': ['newton-cg', 'lbfgs'],
                'max_iter': [250],
            }
        },
        # 'RandomForestClassifier': {
        #     'model': RandomForestClassifier(random_state=models_tuning_seed),
        #     'params': {
        #         "max_depth": [6, 10],
        #         "min_samples_leaf": [1],
        #         "n_estimators": [50, 100],
        #         "max_features": [0.6]
        #     }
        # },
        # 'XGBClassifier': {
        #     'model': XGBClassifier(random_state=models_tuning_seed, verbosity=0),
        #     'params': {
        #         'learning_rate': [0.1],
        #         'n_estimators': [200],
        #         'max_depth': [5, 7],
        #         'lambda':  [10, 100]
        #     }
        # },
        # 'KNeighborsClassifier': {
        #     'model': KNeighborsClassifier(),
        #     'params': {
        #         'n_neighbors' : [5, 7, 9, 11, 13, 15, 25],
        #         'weights' : ['uniform', 'distance'],
        #         'metric' : ['minkowski', 'euclidean', 'manhattan']
        #     }
        # },
        # 'MLPClassifier': {
        #     'model': MLPClassifier(random_state=models_tuning_seed),
        #     'params': {
        #         'hidden_layer_sizes':[(100,), (100,100,), (100,50,100,)],
        #         'activation': ['logistic', 'tanh', 'relu'],
        #         'solver': ['lbfgs', 'sgd', 'adam'],
        #         'learning_rate': ['constant', 'invscaling', 'adaptive']
        #     }
        # }
    }

## Run experiments

### Experiment iteration 1

In [27]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

error_injector = RandomNullsInjectorV2(experiment_seed, columns_to_transform=COLUMNS_TO_TRANSFORM, row_idx_nulls_percentage=0.0,
                                       max_num_columns_to_effect=MAX_NUM_COLUMNS_TO_EFFECT)
models_params_for_tuning = get_models_params_for_tuning(experiment_seed)
# Create a transformed data loader
exp_iter_data_loader = create_experiment_data_loader(data_loader)
exp_iter_data_loader.columns_with_nulls = COLUMNS_TO_TRANSFORM
# preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader, categorical_trimmed=0.3)
preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader)

In [28]:
run_exp_iter_with_models_stress_testing(data_loader=exp_iter_data_loader,
                                        experiment_seed=experiment_seed,
                                        test_set_fraction=TEST_SET_FRACTION,
                                        db_writer_func=db_writer_func,
                                        error_injector=error_injector,
                                        injector_config_lst=INJECTOR_CONFIG_LST,
                                        preprocessor=preprocessor,
                                        models_params_for_tuning=models_params_for_tuning,
                                        metrics_computation_config=metrics_computation_config,
                                        custom_table_fields_dct=custom_table_fields_dct,
                                        with_tuning=True,
                                        save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                        tuned_params_df_path=None,
                                        verbose=True)

2023-04-25 14:18:46 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'error_percentages': '0%,5%,10%,20%,30%,40%,50%',
 'error_type': 'Random Nulls',
 'experiment_iteration': 'Exp_iter_1',
 'injector_config_lst': '[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]',
 'model_init_seed': 100,
 'preprocessing_technique': 'cat: mode, num: median',
 'session_uuid': '0fe1a14e-49c1-4764-962a-b50a2fbed1b7'}




2023-04-25 14:18:47 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([ 4253, 30076, 48047,  1666, 30740, 13830, 34366,  5893, 45012,
            20187, 40178,  7958, 32251,  8493,  8752,  1984,  8898, 10170,
            20852, 19844],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 4253, 30076, 48047,  1666, 30740, 13830, 34366,  5893, 45012,
            20187, 40178,  7958, 32251,  8493,  8752,  1984,  8898, 10170,
            20852, 19844],
           dtype='int64')
error_injector.seed --  101
transformed_X_test:
 MAR         51
MIL         39
ESP         52
MIG         59
DREM        52
NATIVITY    31
DIS         59
DEAR        52
DEYE        54
SEX          0
RAC1P        0
RELP        49
CIT         59
ANC         48
SCHL        50
AGEP         0
dtype: int64
error_injector.seed --  102
transformed_X_test:
 MAR          93
MIL          89
ESP          90
MIG         104
DREM         97
NATIVITY     88
DIS         107
DEAR         94
DEYE        110

2023-04-25 14:19:53 experiment_interface.py INFO    : Models are tuned and saved to a file


...

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/2 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

### Experiment iteration 2

In [29]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

error_injector = RandomNullsInjectorV2(experiment_seed, columns_to_transform=COLUMNS_TO_TRANSFORM, row_idx_nulls_percentage=0.0,
                                       max_num_columns_to_effect=MAX_NUM_COLUMNS_TO_EFFECT)
models_params_for_tuning = get_models_params_for_tuning(experiment_seed)
# Create a transformed data loader
exp_iter_data_loader = create_experiment_data_loader(data_loader)
exp_iter_data_loader.columns_with_nulls = COLUMNS_TO_TRANSFORM
# preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader, categorical_trimmed=0.3)
preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader)

In [30]:
run_exp_iter_with_models_stress_testing(data_loader=exp_iter_data_loader,
                                        experiment_seed=experiment_seed,
                                        test_set_fraction=TEST_SET_FRACTION,
                                        db_writer_func=db_writer_func,
                                        error_injector=error_injector,
                                        injector_config_lst=INJECTOR_CONFIG_LST,
                                        preprocessor=preprocessor,
                                        models_params_for_tuning=models_params_for_tuning,
                                        metrics_computation_config=metrics_computation_config,
                                        custom_table_fields_dct=custom_table_fields_dct,
                                        with_tuning=True,
                                        save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                        tuned_params_df_path=None,
                                        verbose=True)

2023-04-25 14:23:08 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'error_percentages': '0%,5%,10%,20%,30%,40%,50%',
 'error_type': 'Random Nulls',
 'experiment_iteration': 'Exp_iter_2',
 'injector_config_lst': '[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]',
 'model_init_seed': 200,
 'preprocessing_technique': 'cat: mode, num: median',
 'session_uuid': '0fe1a14e-49c1-4764-962a-b50a2fbed1b7'}




2023-04-25 14:23:10 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([20518, 20570, 38530, 39945,  1713, 19318, 15625, 36291, 26020,
            18954,  7039, 40519, 42300,  5620, 39590, 39048, 33565, 18881,
            43190,  5750],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([20518, 20570, 38530, 39945,  1713, 19318, 15625, 36291, 26020,
            18954,  7039, 40519, 42300,  5620, 39590, 39048, 33565, 18881,
            43190,  5750],
           dtype='int64')
error_injector.seed --  201
transformed_X_test:
 MAR         43
MIL         53
ESP         50
MIG         48
DREM        38
NATIVITY    34
DIS         42
DEAR        50
DEYE        44
SEX          0
RAC1P        0
RELP        40
CIT         54
ANC         50
SCHL        44
AGEP         0
dtype: int64
error_injector.seed --  202
transformed_X_test:
 MAR         101
MIL          94
ESP          86
MIG          96
DREM         98
NATIVITY     79
DIS          96
DEAR        113
DEYE         96

2023-04-25 14:24:16 experiment_interface.py INFO    : Models are tuned and saved to a file


...

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/2 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

In [54]:
client.close()